In [1]:
import numpy as np
import time
import cudf
import cuspatial
%load_ext snakeviz

In [2]:
import pickle

with open('data/trajectories_extracted.p','rb') as f:
    trajectories = pickle.load(f)

In [3]:
def convert_latlon_2_image_for_vis_wrongway_intersection(lat,long):

    x_anchor = 436.369
    y_anchor = 126.613
    long_anchor = -90.666714
    lat_anchor = 42.492466

    long_1 = -90.663183
    lat_1 = 42.491671
    x_1 = 971.026
    y_1 = 291.955

    long_2_x_rate = (x_1-x_anchor)/(long_1-long_anchor)
    lat_2_y_rate = (y_1-y_anchor)/(lat_1-lat_anchor)

    y = (lat-lat_anchor)*lat_2_y_rate+y_anchor
    x = (long-long_anchor)*long_2_x_rate+x_anchor

    return x, y

def convert_traj_latlon_2_image_wrongway_intersection(traj):

    traj_image = []
    for pt in traj:
        lat = pt[0]
        long = pt[1]
        x,y = convert_latlon_2_image_for_vis_wrongway_intersection(lat,long)
        traj_image.append((x,y))

    return np.asarray(traj_image)

def traj_filter_num_points(trajectories,num_min):
    max_length = 0
    min_length = 999
    lengths = []
    long_trajectories = []
    for traj in trajectories:
        if len(traj)>=num_min:
            long_trajectories.append(traj)
            if len(traj)<min_length:
                min_length = len(traj)
            lengths.append(len(traj))
        if len(traj) > max_length:
            max_length = len(traj)
    print("the longest traj length is: ",max_length)
    print("the shortest traj length is: ",min_length)
    print("the average traj length is: ",int(np.mean(lengths)))
    return long_trajectories

def pick_traj_with_label(label,labels,trajectories):
    selected_trajectories = []
    for i in range(len(labels)):
        if labels[i] == label:
            selected_trajectories.append(trajectories[i])
    return selected_trajectories  

In [4]:
print('Filtering based on number of points ...')
trajectories = traj_filter_num_points(trajectories,num_min=30)
print('{} left after filtering based on number of points'.format(len(trajectories)))

trajectories = [convert_traj_latlon_2_image_wrongway_intersection(trajectory) for trajectory in trajectories]

Filtering based on number of points ...
the longest traj length is:  961
the shortest traj length is:  30
the average traj length is:  87
10070 left after filtering based on number of points


In [7]:
start = time.time()
cnt = []
for traj in trajectories:
    cnt.append(len(traj))
cnt = np.asarray(cnt)

trajs = np.concatenate([np.asarray(traj) for traj in trajectories],0)

x_s = trajs[:,0]
y_s = trajs[:,1]

pnt_x=cudf.Series(x_s)
pnt_y=cudf.Series(y_s)
cnt=cudf.Series(cnt)

In [8]:
%%snakeviz
start1 = time.time()
dist=cuspatial.directed_hausdorff_distance(pnt_x,pnt_y,cnt)
end1 = time.time() 

 
*** Profile stats marshalled to file '/tmp/tmpnk95ax_0'. 
Embedding SnakeViz in the notebook...


In [ ]:
print('\nHausdorff distance calculation by Cuspatial in GPU takes: \n  {0:.3f} seconds (~1500X)'.format(end1 - start1))
urt=dist.data.to_array()
dmatrix = np.reshape(urt,(len(cnt),len(cnt)))
end = time.time()
print('\nThe complete Dmatrix calculation including data transitioning and transforming takes: \n  {0:.3f} seconds (~1400X)'.format(end - start)) 

In [7]:
import cudf.core.column as column
c_x=column.as_column(np.array(x_s,dtype=np.float64))
c_y=column.as_column(np.array(y_s,dtype=np.float64))
c_cnt=column.as_column(np.array(cnt,dtype=np.int32))

start_c = time.time()
dist=cuspatial.directed_hausdorff_distance_column(c_x,c_y,c_cnt)
end_c = time.time()
print('\nHausdorff distance calculation by Cuspatial in GPU takes: \n  {0:.3f} seconds (~1500X)'.format(end_c - start_c))


Hausdorff distance calculation by Cuspatial in GPU takes: 
  12.609 seconds (~1500X)
